# Bloodhound Playbook - Domain Admin Sessions
-----------------------------------------


## Import Libraries

In [15]:
from py2neo import Graph

## Initialize BloodHound neo4j Database Connection

In [16]:
g = Graph("bolt://127.0.0.1:7687", auth=("neo4j", "neo4j"))
g

<Graph database=<Database uri='bolt://127.0.0.1:7687' secure=False user_agent='py2neo/4.2.0 neobolt/1.7.15 Python/3.7.4-final-0 (win32)'> name='data'>

## Retrieve All Domain Admin Account Sessions

Now we’ll construct the Cypher query that return all DA account sessions

In [17]:
da_sessions_df = g.run("""
MATCH (n:User)-[:MemberOf]->(g:Group {name:"DOMAIN ADMINS@TESTLAB.NET"})
MATCH p = (c:Computer)-[:HasSession]->(n)
return distinct n.name, count(p) AS DA_sessions
ORDER BY DA_sessions DESC
""").to_data_frame()

In [ ]:
da_sessions_df

## Retrieve Domain Admin Sessions To Non-Domain Controllers

In [19]:
non_dc_df = g.run("""
OPTIONAL MATCH (c:Computer)-[:MemberOf]->(t:Group)
WHERE NOT t.name = "DOMAIN CONTROLLERS@TESTLAB.NET"
WITH c as NonDC
MATCH p=(NonDC)-[:HasSession]->(n:User)-[:MemberOf]->
(g:Group {name:"DOMAIN ADMINS@TESTLAB.NET"})
RETURN DISTINCT (n.name) as Username, COUNT(DISTINCT(NonDC)) as Connections
ORDER BY COUNT(DISTINCT(NonDC)) DESC
""").to_data_frame()

In [ ]:
non_dc_df